In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .appName("windows fuctions")\
    .getOrCreate()
print(spark)

25/03/20 19:58:04 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 192.168.163.68 instead (on interface wlp0s20f3)
25/03/20 19:58:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 19:58:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [6]:
window_Spec = Window.partitionBy("department").orderBy("salary")

In [7]:
df.withColumn("row_number",row_number().over(window_Spec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |2         |
|Robert       |Sales     |4100  |3         |
|Saif         |Sales     |4100  |4         |
|Michael      |Sales     |4600  |5         |
+-------------+----------+------+----------+



In [10]:
df.withColumn("row_number",row_number().over(Window.orderBy("salary"))).show()

25/03/20 20:11:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/20 20:11:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/20 20:11:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        Kumar| Marketing|  2000|         1|
|        James|     Sales|  3000|         2|
|        Maria|   Finance|  3000|         3|
|        James|     Sales|  3000|         4|
|         Jeff| Marketing|  3000|         5|
|        Scott|   Finance|  3300|         6|
|          Jen|   Finance|  3900|         7|
|       Robert|     Sales|  4100|         8|
|         Saif|     Sales|  4100|         9|
|      Michael|     Sales|  4600|        10|
+-------------+----------+------+----------+



#### for rank()

In [12]:
from pyspark.sql.functions import rank

In [13]:
df.withColumn("rank",rank().over(window_Spec)).show(truncate = False)

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |1   |
|Scott        |Finance   |3300  |2   |
|Jen          |Finance   |3900  |3   |
|Kumar        |Marketing |2000  |1   |
|Jeff         |Marketing |3000  |2   |
|James        |Sales     |3000  |1   |
|James        |Sales     |3000  |1   |
|Robert       |Sales     |4100  |3   |
|Saif         |Sales     |4100  |3   |
|Michael      |Sales     |4600  |5   |
+-------------+----------+------+----+



#### dense_rank()


In [14]:
from pyspark.sql.functions import dense_rank

In [15]:
df.withColumn("dense-rank",dense_rank().over(window_Spec)).show(truncate = False)

+-------------+----------+------+----------+
|employee_name|department|salary|dense-rank|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |1         |
|Robert       |Sales     |4100  |2         |
|Saif         |Sales     |4100  |2         |
|Michael      |Sales     |4600  |3         |
+-------------+----------+------+----------+



#### percentage_rank()


In [16]:
from pyspark.sql.functions import percent_rank

In [17]:
df.withColumn("percent_rank",percent_rank().over(window_Spec)).show(truncate =False)

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|Maria        |Finance   |3000  |0.0         |
|Scott        |Finance   |3300  |0.5         |
|Jen          |Finance   |3900  |1.0         |
|Kumar        |Marketing |2000  |0.0         |
|Jeff         |Marketing |3000  |1.0         |
|James        |Sales     |3000  |0.0         |
|James        |Sales     |3000  |0.0         |
|Robert       |Sales     |4100  |0.5         |
|Saif         |Sales     |4100  |0.5         |
|Michael      |Sales     |4600  |1.0         |
+-------------+----------+------+------------+



#### ntile(-) - bucketing dividing the data into equal no of groups.

In [18]:
from pyspark.sql.functions import ntile

In [20]:
df.withColumn("bucket_number",ntile(2).over(window_Spec)).show(truncate = False)

+-------------+----------+------+-------------+
|employee_name|department|salary|bucket_number|
+-------------+----------+------+-------------+
|Maria        |Finance   |3000  |1            |
|Scott        |Finance   |3300  |1            |
|Jen          |Finance   |3900  |2            |
|Kumar        |Marketing |2000  |1            |
|Jeff         |Marketing |3000  |2            |
|James        |Sales     |3000  |1            |
|James        |Sales     |3000  |1            |
|Robert       |Sales     |4100  |1            |
|Saif         |Sales     |4100  |2            |
|Michael      |Sales     |4600  |2            |
+-------------+----------+------+-------------+



#### cume_dist()

In [21]:
from pyspark.sql.functions import cume_dist

In [22]:
df.withColumn("cumulative distribution",cume_dist().over(window_Spec)).show(truncate = False)

[Stage 20:==================================================>       (7 + 1) / 8]

+-------------+----------+------+-----------------------+
|employee_name|department|salary|cumulative distribution|
+-------------+----------+------+-----------------------+
|Maria        |Finance   |3000  |0.3333333333333333     |
|Scott        |Finance   |3300  |0.6666666666666666     |
|Jen          |Finance   |3900  |1.0                    |
|Kumar        |Marketing |2000  |0.5                    |
|Jeff         |Marketing |3000  |1.0                    |
|James        |Sales     |3000  |0.4                    |
|James        |Sales     |3000  |0.4                    |
|Robert       |Sales     |4100  |0.8                    |
|Saif         |Sales     |4100  |0.8                    |
|Michael      |Sales     |4600  |1.0                    |
+-------------+----------+------+-----------------------+



#### lag() lead()

In [23]:
from pyspark.sql.functions import lag,lead

In [28]:
df.withColumn("lag",lag(df.salary,1).over(window_Spec)).show(truncate =False)

+-------------+----------+------+----+
|employee_name|department|salary|lag |
+-------------+----------+------+----+
|Maria        |Finance   |3000  |NULL|
|Scott        |Finance   |3300  |3000|
|Jen          |Finance   |3900  |3300|
|Kumar        |Marketing |2000  |NULL|
|Jeff         |Marketing |3000  |2000|
|James        |Sales     |3000  |NULL|
|James        |Sales     |3000  |3000|
|Robert       |Sales     |4100  |3000|
|Saif         |Sales     |4100  |4100|
|Michael      |Sales     |4600  |4100|
+-------------+----------+------+----+



In [31]:
df.withColumn("lead",lead(df.salary).over(window_Spec)).show(truncate = False)

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |3300|
|Scott        |Finance   |3300  |3900|
|Jen          |Finance   |3900  |NULL|
|Kumar        |Marketing |2000  |3000|
|Jeff         |Marketing |3000  |NULL|
|James        |Sales     |3000  |3000|
|James        |Sales     |3000  |4100|
|Robert       |Sales     |4100  |4100|
|Saif         |Sales     |4100  |4600|
|Michael      |Sales     |4600  |NULL|
+-------------+----------+------+----+



In [34]:
windowSpecAgg  = Window.partitionBy("department")
from pyspark.sql.functions import col,avg,sum,min,max,row_number 
df.withColumn("row",row_number().over(window_Spec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()

+----------+------+-----+----+----+
|department|   avg|  sum| min| max|
+----------+------+-----+----+----+
|   Finance|3400.0|10200|3000|3900|
| Marketing|2500.0| 5000|2000|3000|
|     Sales|3760.0|18800|3000|4600|
+----------+------+-----+----+----+



In [37]:
spark.stop()